# 使用GPU训练--方法2

In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torch import nn
import torch
import time


# 定义训练的设备
device = torch.device("cuda") # cuda 或者 GPu

# 1. 准备训练数据集
train_data = torchvision.datasets.CIFAR10(
    root='./datasets',
      train=True, 
      transform=torchvision.transforms.ToTensor(), 
      download=True
)


# 2. 准备测试数据集
test_data = torchvision.datasets.CIFAR10(
    root='./datasets',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

train_data_size = len(train_data) # 训练数据集的大小 50000张
test_data_size = len(test_data)   # 测试数据集的大小 10000张


# 3. 利用Dataloader 加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


# 4. 搭建神经网络
# 因为 CIRAR10 是一个 10 分类的数据集，所以输出的类别是 10 个
# 这个可以单独放到一个文件中，比如model.py，然后导入这个包
class TuDui(nn.Module):
    def __init__(self):
        super(TuDui, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        output = self.model(x)
        return output


# 5. 创建网络模型
tudui = TuDui()
tudui = tudui.to(device)  # 事实上 写成tudui.to(device)就可以了，不用赋值，损失函数也是这样的
    

# 6. 创建损失函数
loss_function = nn.CrossEntropyLoss()  # CrossEntropyLoss这个损失常用于分类问题
loss_function = loss_function.to(device)   # 使用GPU训练（损失函数也可以调用）


# 7. 创建优化器
learning_rate = 0.001 # 学习率， 这个通常单独作为一个变量传入优化器,  便于修改
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate, )

In [ ]:
# 8. 设置训练网络的一些参数
total_train_step = 0  # 记录训练的总次数
total_test_step = 0   # 记录测试的总次数
epoch = 10            # 训练的轮数

start_time = time.time()  # 记录开始训练的时间
for i in range(epoch):
    print("-----------------第 {} 轮训练开始-----------------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        img, target = data
        img = img.to(device)  # 调用GPU
        target = target.to(device) # 调用GPU
        output = tudui(img) # 投喂到神经网络中得到一个输出
        loss = loss_function(output, target)  # 计算损失（预测输出和真实值之间的关系）

        # 开始优化（利用优化器优化模型）
        optimizer.zero_grad()  # 优化之前一定要先把梯度清零
        loss.backward()   # 利用损失函数的反向传播，得到每个参数节点的梯度
        optimizer.step()  # 对其中的参数进行一个优化

        total_train_step = total_train_step + 1

        if total_train_step % 100 == 0:
            end_time = time.time()
            print('time: {:.4f}'.format(end_time-start_time))
            print("训练次数：{}，Loss：{:.4f}".format(total_train_step, loss.item()))
        

    # 上边是完整的一轮训练流程
    # 当一轮训练结束后，如何来判断训练的结果到底怎么样，有没有达到预期，需要对训练的结果进行测试
    # 根据测试数据集上的正确率/损失，来判断模型有没有训练好
    # 测试的过程就不要调优了，需要根据现有模型来测试
    total_test_loss = 0  # 本轮测试的总损失
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            img, target = data
            img = img.to(device)
            target = target.to(device)
            output = tudui(img) # 将测试数据投喂到网络中
            loss = loss_function(output,target)  # 计算模型输出与真实值之间的误差
            total_test_loss = total_test_loss + loss.item()
    print("整体测试集上的Loss:{:.4f}.".format(total_test_loss))